In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import tensorflow as tf

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

In [2]:
# load data: digits 5 to 9, but still label with 0 to 4, 
# because TensorFlow expects label's integers from 0 to n_classes-1.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

X_train2_full = mnist.train.images[mnist.train.labels >= 5]
y_train2_full = mnist.train.labels[mnist.train.labels >= 5] - 5
X_valid2_full = mnist.validation.images[mnist.validation.labels >= 5]
y_valid2_full = mnist.validation.labels[mnist.validation.labels >= 5] - 5
X_test2 = mnist.test.images[mnist.test.labels >= 5]
y_test2 = mnist.test.labels[mnist.test.labels >= 5] - 5

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# we want to keep only 100 instances per class in the training set 
# and let's keep only 30 instances per class in the validation set
# tesing set is already loaded above
def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)
    return np.concatenate(Xs), np.concatenate(ys)

X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_valid2, y_valid2 = sample_n_instances_per_class(X_valid2_full, y_valid2_full, n=30)

## 3.1 Train Softmax Only

In [51]:
reset_graph()

with tf.Session() as sess:
    restore_saver = tf.train.import_meta_graph("./Models/yen/mnist_5.ckpt.meta")
    restore_saver.restore(sess, tf.train.latest_checkpoint('./Models/yen/'))
#     sess.run(tf.global_variables_initializer()) # initialize the weights 
#     sess.run(tf.local_variables_initializer()) # initialize the local variables hidden in the tf.metrics.recallmethod.

    all_vars = tf.trainable_variables()
    for v in all_vars:

        print("%s with value %s" % (v.name, sess.run(v)))

INFO:tensorflow:Restoring parameters from ./Models/yen/mnist_5.ckpt
Fully_Connected_Layer_1/kernel:0 with value [[ 0.03314166 -0.09055354 -0.08280303 ...,  0.04810002 -0.07048665
   0.03521701]
 [ 0.06279289 -0.02778894  0.01664921 ..., -0.01901775  0.06809777
   0.04352102]
 [-0.05096035  0.03554505 -0.05947971 ..., -0.10324872  0.06255207
   0.00371845]
 ..., 
 [ 0.01253158  0.10347325  0.03930723 ..., -0.00621642 -0.05332955
   0.08313946]
 [-0.02647174  0.06996583  0.04140012 ...,  0.00750363 -0.09673171
  -0.00074043]
 [ 0.05517064  0.02086869  0.01615661 ...,  0.08531171  0.06016324
   0.06593282]]
Fully_Connected_Layer_1/bias:0 with value [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  

In [22]:
reset_graph()

no_update_tolerant = 20

no_update_progress = 0
current_best_accuracy = 0.0
current_best_loss = 0.0

with tf.Session() as sess:
    restore_saver = tf.train.import_meta_graph("./Models/yen/mnist_5.ckpt.meta") # load meta graph
    restore_saver.restore(sess, tf.train.latest_checkpoint('./Models/yen/')) # restore weights
#     sess.run(tf.global_variables_initializer()) # initialize the weights 
#     sess.run(tf.local_variables_initializer()) # initialize the local variables hidden in the tf.metrics.recallmethod.

    """
    Graph Define Step
    
    1. Restore the variable about the component in graph which help to retrieve and assign value
    """
    graph = tf.get_default_graph() # get original graph
    X = graph.get_tensor_by_name("Input_X:0") # assign placeholder variables with input X
    y = graph.get_tensor_by_name("Input_Label:0")# assign placeholder variables with input Label
#     mode = graph.get_tensor_by_name("Mode:0")# assign placeholder variables with input Mode
    loss = graph.get_tensor_by_name("Loss:0") # assign variables about loss
    Y_proba = graph.get_tensor_by_name("Softmax_Layer/Softmax:0") # assign variables about loss
    accuracy = graph.get_tensor_by_name("Accuracy:0") # assign variables  about accuracy
#     optimizer = graph.get_operation_by_name("Adam")# assign variables about optimizer which is adam optimizer
#     print(type(optimizer))
#     print(Y_proba.op.inputs[0])
    """
    2. Get the softmax layer which is what we would like to train
    """
    output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="Softmax_Layer") # get softmax layer by scope name
    print(output_layer_vars)
    
    """
    3. Define the optimizer and assign to only update on softmax_layer which defined above
    """
    optimizer = tf.train.AdamOptimizer(0.0017, name="Op")
    training_op = optimizer.minimize(loss, var_list=output_layer_vars) # minimize the loss and only update certern variables
    tf.variables_initializer([optimizer.get_slot(var, name) for name in optimizer.get_slot_names() for var in output_layer_vars])
    tf.variables_initializer(list(optimizer._get_beta_accumulators()))
    sess.run(tf.global_variables_initializer())
    """
    Training Step
    """
    for epochs in range(1000): # training on 1000 epochs
        _, train_loss = sess.run([training_op, loss], feed_dict={X:X_train2, y:y_train2}) # feed with training variables
        valid_loss, valid_accuracy = sess.run([loss, accuracy], feed_dict={X:X_valid2, y:y_valid2}) # feed with validation variables
        test_loss ,  test_accuracy = sess.run([loss, accuracy], feed_dict={X:X_test2, y:y_test2}) # feed with testing variables
        
        print("{:4d} epochs, Validation loss:{:.5f}, Best loss:{:.5f}, Accuracy:{:.5f}/ {:.5f}".format(
            epochs+1,
            valid_loss,
            current_best_loss,                                                                                       
            valid_accuracy,
            test_accuracy
        ))
        
        if valid_accuracy > current_best_accuracy: # if better
            current_best_accuracy = valid_accuracy # keep record
        else:                                      # if not better
            no_update_progress += 1                # record on no update

        if valid_loss < current_best_loss: # if better
            current_best_loss = valid_loss # keep record            
            
        if no_update_progress == no_update_tolerant: break # stop training
        
        

INFO:tensorflow:Restoring parameters from ./Models/yen/mnist_5.ckpt
[<tf.Variable 'Softmax_Layer/kernel:0' shape=(128, 5) dtype=float32_ref>, <tf.Variable 'Softmax_Layer/bias:0' shape=(5,) dtype=float32_ref>]
   1 epochs, Validation loss:1.63157, Best loss:0.00000, Accuracy:0.08000/ 0.23802
   2 epochs, Validation loss:1.62380, Best loss:0.00000, Accuracy:0.10667/ 0.25797
   3 epochs, Validation loss:1.61584, Best loss:0.00000, Accuracy:0.13333/ 0.27669
   4 epochs, Validation loss:1.60770, Best loss:0.00000, Accuracy:0.18000/ 0.29850
   5 epochs, Validation loss:1.59941, Best loss:0.00000, Accuracy:0.23333/ 0.32010
   6 epochs, Validation loss:1.59098, Best loss:0.00000, Accuracy:0.25333/ 0.33944
   7 epochs, Validation loss:1.58245, Best loss:0.00000, Accuracy:0.29333/ 0.35569
   8 epochs, Validation loss:1.57384, Best loss:0.00000, Accuracy:0.35333/ 0.37564
   9 epochs, Validation loss:1.56519, Best loss:0.00000, Accuracy:0.38000/ 0.39066
  10 epochs, Validation loss:1.55651, Best l